In [ ]:
from QAOA_simulator import get_simulator, get_expectation, get_result
import matplotlib.pyplot as plt
import networkx as nx
import qokit.maxcut as mc
import numpy as np
from QAOA_proxy import QAOA_proxy
from QAOA_paper_proxy import QAOA_paper_proxy
import os

"""Plots a heat map for the given data

Parameters
----------
xs : np.ndarray
    x-coordinates (no duplication)
ys : np.ndarray
    y-coordinates (no duplication)
data : np.ndarray (2-dimensional)
    data[x][y] will be the heat at (x, y)
"""

def plot_heat_map(xs: np.ndarray, ys: np.ndarray, data: np.ndarray, title: str, xlabel: str, ylabel: str, num_labels: int = 10, vmin=None, vmax=None) -> plt.Figure:
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    # Create the heatmap
    cax = ax.imshow(data.T, aspect="auto", origin='lower', vmin=vmin, vmax=vmax, cmap='viridis')

    # Determine the number of labels and the step size
    x_step = len(xs) // num_labels
    y_step = len(ys) // num_labels

    # Select equally spaced indices
    x_indices = np.arange(0, len(xs), x_step)
    y_indices = np.arange(0, len(ys), y_step)

    # Handle cases where the last label might be skipped
    if x_indices[-1] != len(xs) - 1:
        x_indices = np.append(x_indices, len(xs) - 1)
    if y_indices[-1] != len(ys) - 1:
        y_indices = np.append(y_indices, len(ys) - 1)

    # Set the ticks and tick labels
    ax.set_xticks(x_indices)
    ax.set_xticklabels([f"{xs[i]:.2f}" for i in x_indices])
    ax.set_yticks(y_indices)
    ax.set_yticklabels([f"{ys[i]:.2f}" for i in y_indices])
    ax.set_ylim(ax.get_ylim()[::-1])

    # Adds a color bar
    fig.colorbar(cax)
    return fig


def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> np.ndarray:
    expectations = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)

    return expectations


def collect_parameter_data_proxy_less_dumb(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> np.ndarray:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits)

    return expectations


def collect_parameter_data_proxy(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> np.ndarray:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits)

    return expectations

gammas = np.linspace(-np.pi, np.pi, 50)
betas = np.linspace(-2 * np.pi, 2 * np.pi, 50)

all_expectations = []

for n in range(5, 16):
    N = n
    G = nx.erdos_renyi_graph(N, 0.5, seed=18)
    M = G.number_of_edges()

    qaoa_path = f"data_for_Expectation_Heatmap/QAOA_N={N}_M={M}.npy"
    paper_proxy_path = f"data_for_Expectation_Heatmap/Paper_Proxy_N={N}_M={M}.npy"
    our_proxy_path = f"data_for_Expectation_Heatmap/Our_Proxy_N={N}_M={M}.npy"

    if os.path.exists(qaoa_path):
        expectations = np.load(qaoa_path)
    else:
        expectations = collect_parameter_data(G, gammas, betas)
        np.save(qaoa_path, expectations)

    if os.path.exists(paper_proxy_path):
        expectation_proxies = np.load(paper_proxy_path)
    else:
        expectation_proxies = collect_parameter_data_proxy(M, N, gammas, betas)
        np.save(paper_proxy_path, expectation_proxies)

    if os.path.exists(our_proxy_path):
        expectation_proxies_less_dumb = np.load(our_proxy_path)
    else:
        expectation_proxies_less_dumb = collect_parameter_data_proxy_less_dumb(M, N, gammas, betas)
        np.save(our_proxy_path, expectation_proxies_less_dumb)

    all_expectations.extend(expectations.flatten())
    all_expectations.extend(expectation_proxies.flatten())
    all_expectations.extend(expectation_proxies_less_dumb.flatten())

global_min = min(all_expectations)
global_max = max(all_expectations)

for n in range(5, 16):
    N = n
    G = nx.erdos_renyi_graph(N, 0.5, seed=18)
    M = G.number_of_edges()

    expectations = np.load(f"data_for_Expectation_Heatmap/QAOA_N={N}_M={M}.npy")
    expectation_proxies = np.load(f"data_for_Expectation_Heatmap/Paper_Proxy_N={N}_M={M}.npy")
    expectation_proxies_less_dumb = np.load(f"data_for_Expectation_Heatmap/Our_Proxy_N={N}_M={M}.npy")

    _ = plot_heat_map(gammas, betas, expectations, f"True QAOA Expectation (N={N},M={M})", "Gamma", "Beta", vmin=global_min, vmax=global_max)
    _ = plot_heat_map(gammas, betas, expectation_proxies, f"Expectation Proxies from Paper (N={N},M={M})", "Gamma", "Beta", vmin=global_min, vmax=global_max)
    _ = plot_heat_map(gammas, betas, expectation_proxies_less_dumb, f"Expectation Proxies from us (N={N},M={M})", "Gamma", "Beta", vmin=global_min, vmax=global_max)

    plt.show()

In [ ]:
from QAOA_paper_proxy import QAOA_paper_proxy
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np

"""
Paper proxy only
"""

def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> np:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits)

    return expectations

gammas = np.linspace(-np.pi, np.pi, 50)
betas = np.linspace(-2 * np.pi, 2 * np.pi, 50)

# Run experiments for each N
for n in range(2, 16):
    m = int(n * (n - 1) / 4)
    if m < n:
        m = n
    expectations = collect_parameter_data(m, n, gammas, betas)
    plot_heat_map(gammas, betas, expectations, f"Expectation Proxy from paper (N={n} M={m})", "Gamma", "Beta")
    plt.show()

    # Save data
    np.save(f"data_for_Expectation_Heatmap/Paper_Proxy_N={n}_M={m}.npy", expectations)

In [ ]:
from QAOA_proxy import QAOA_proxy, QAOA_proxy_run
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np

"""
Our proxy only
"""

def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> np:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits)

    return expectations

gammas = np.linspace(-np.pi, np.pi, 50)
betas = np.linspace(-2 * np.pi, 2 * np.pi, 50)

# Run experiments for each N
for n in range(5, 16):
    m = int(n * (n - 1) / 4)
    if m < n:
        m = n
    expectations = collect_parameter_data(m, n, gammas, betas)
    plot_heat_map(gammas, betas, expectations, f"Expectation Proxy from us (N={n} M={m})", "Gamma", "Beta")
    plt.show()

    result = QAOA_proxy_run(m, n, 1, np.array([0.01]), np.array([0.01]))
    print(result)

    # Save data
    np.save(f"data_for_Expectation_Heatmap/Our_Proxy_N={n}_M={m}.npy", expectations)
